# Tutorial: Classificação Não Supervisionada de Áreas Agrícolas e Florestais usando CNN

Este notebook apresenta um guia passo a passo sobre como usar Redes Neurais Convolucionais (CNN) para identificar e classificar automaticamente áreas agrícolas e florestais usando imagens de sensoriamento remoto do Sentinel-2.

## Conteúdo:
1. Introdução ao Sensoriamento Remoto para Agricultura e Floresta
2. Preparação dos Dados
3. Implementação do Modelo
4. Treinamento e Validação
5. Visualização e Análise dos Resultados

## 1. Introdução

### 1.1 Sensoriamento Remoto na Agricultura e Floresta
O sensoriamento remoto é uma ferramenta fundamental para monitoramento agrícola e florestal, permitindo:
- Identificação de diferentes culturas agrícolas
- Monitoramento do desenvolvimento da vegetação
- Detecção de desmatamento
- Análise de saúde da vegetação
- Identificação de áreas de expansão agrícola

### 1.2 Por que usar CNN?
CNNs são especialmente eficazes para análise de imagens agrícolas e florestais porque:
- Podem identificar padrões complexos de cultivo
- Reconhecem diferentes estágios de crescimento da vegetação
- Diferenciam tipos de cobertura florestal
- São robustas a variações sazonais

### 1.3 Abordagem Não Supervisionada
A abordagem não supervisionada é particularmente útil no contexto agrícola e florestal porque:
- Não requer dados rotulados extensivos
- Pode descobrir padrões naturais de uso do solo
- Adapta-se a diferentes regiões e tipos de vegetação

## 2. Preparação do Ambiente

Primeiro, vamos importar as bibliotecas necessárias:

In [3]:
import sys
sys.path.append('..')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from src.data.tiff_loader import TiffLoader
from src.models.unsupervised_cnn import UnsupervisedCNN
import mlflow
import seaborn as sns

plt.style.use('seaborn')
%matplotlib inline

2025-04-01 15:49:05.753029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743533345.844836   25114 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743533345.869664   25114 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743533346.076042   25114 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743533346.076079   25114 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743533346.076082   25114 computation_placer.cc:177] computation placer alr

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

## 3. Entendendo os Dados do Sentinel-2 para Agricultura e Floresta

### 3.1 Bandas Espectrais Relevantes

As bandas selecionadas são especialmente úteis para análise agrícola e florestal:

- **B8 (NIR - 842nm)**:
  - Alta reflectância em vegetação saudável
  - Excelente para avaliar biomassa
  - Diferencia tipos de culturas

- **B4 (Red - 665nm)**:
  - Absorção por clorofila
  - Indica estresse da vegetação
  - Útil para distinguir solo exposto

- **B11 (SWIR - 1610nm)**:
  - Sensível ao conteúdo de água
  - Ajuda a distinguir tipos de culturas
  - Útil para identificar áreas irrigadas

### 3.2 Carregando os Dados

In [ ]:
# Inicializar o loader
loader = TiffLoader(
    train_val_dir='../data/train',
    prediction_dir='../data/predict',
    patch_size=64
)

# Carregar dados de treino e validação
train_ds, val_ds = loader.load_train_val_data(val_split=0.2)

# Preparar os datasets
train_ds = loader.prepare_dataset(train_ds, batch_size=32, shuffle=True, augment=True)
val_ds = loader.prepare_dataset(val_ds, batch_size=32, shuffle=False)

### 3.3 Visualizando Padrões Agrícolas e Florestais

Vamos visualizar alguns exemplos e aprender a identificar diferentes padrões:

In [ ]:
def plot_patches(dataset, num_patches=5):
    plt.figure(figsize=(15, 3))
    for i, batch in enumerate(dataset.take(1)):
        for j in range(num_patches):
            plt.subplot(1, num_patches, j+1)
            rgb = batch[j].numpy()
            plt.imshow(rgb)
            plt.axis('off')
            plt.title(f'Patch {j+1}')
    plt.tight_layout()

plot_patches(train_ds)

## 4. Implementação e Treinamento do Modelo

### 4.1 Arquitetura do Modelo

Nossa CNN é otimizada para detectar:
- Padrões regulares de cultivo agrícola
- Texturas características de florestas
- Diferentes estágios de crescimento
- Áreas de transição agricultura-floresta

In [ ]:
# Inicializar o modelo
model = UnsupervisedCNN(
    input_shape=(64, 64, 3),
    n_clusters=5,  # Ajuste baseado nos tipos esperados de cobertura
    latent_dim=128,
    experiment_name='agricultura_floresta_classificacao'
)

# Converter dataset para numpy array
X_train = np.concatenate([batch.numpy() for batch in train_ds], axis=0)

# Treinar o modelo
model.train(
    X=X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    model_params={'description': 'Classificação Agricultura-Floresta'}
)

## 5. Interpretação dos Resultados

### 5.1 Padrões Esperados

Ao analisar os clusters, procure por:

**Áreas Agrícolas:**
- Padrões geométricos regulares
- Variações de intensidade por estágio de cultivo
- Bordas bem definidas entre talhões

**Áreas Florestais:**
- Textura mais rugosa e irregular
- Alta resposta no NIR (B8)
- Padrão mais homogêneo em grandes áreas

**Transições e Casos Especiais:**
- Áreas de regeneração florestal
- Sistemas agroflorestais
- Agricultura em diferentes estágios

In [ ]:
# Fazer predições
predictions = model.predict(X_train)

# Visualizar exemplos de cada cluster
plt.figure(figsize=(15, 3*model.n_clusters))
for cluster in range(model.n_clusters):
    cluster_samples = X_train[predictions == cluster][:5]
    for j, sample in enumerate(cluster_samples):
        plt.subplot(model.n_clusters, 5, cluster*5 + j + 1)
        plt.imshow(sample)
        plt.axis('off')
        if j == 0:
            plt.ylabel(f'Cluster {cluster}')
plt.tight_layout()

## 6. Categorização dos Clusters

Vamos categorizar os clusters com foco em agricultura e floresta:

In [ ]:
from src.models.cluster_categorizer import ClusterCategorizer

categorizer = ClusterCategorizer(n_clusters=model.n_clusters)

# Exemplo de categorias específicas para agricultura e floresta
categorias_exemplo = [
    ('floresta_densa', 'Floresta madura com dossel fechado'),
    ('agricultura_estabelecida', 'Culturas em estágio avançado'),
    ('agricultura_inicial', 'Áreas recém plantadas ou em preparo'),
    ('floresta_regeneracao', 'Áreas em processo de regeneração florestal'),
    ('agroflorestal', 'Sistemas que combinam agricultura e floresta')
]

# Categorizar clusters
for i, (categoria, descricao) in enumerate(categorias_exemplo):
    if i < model.n_clusters:
        categorizer.categorize_cluster(
            cluster_id=i,
            category=categoria,
            description=descricao,
            confidence=0.8
        )

# Visualizar resultados
categorizer.plot_cluster_samples(X_train, predictions)

## 7. Dicas para Interpretação

### 7.1 Características Espectrais

**Floresta:**
- Alta reflectância no NIR (B8)
- Baixa reflectância no vermelho (B4)
- Valores intermediários no SWIR (B11)

**Agricultura:**
- Variação sazonal na reflectância
- Padrões distintos por tipo de cultura
- Resposta espectral varia com estágio de crescimento

### 7.2 Padrões Espaciais

**Floresta:**
- Textura rugosa
- Bordas naturais irregulares
- Variação gradual de densidade

**Agricultura:**
- Formas geométricas regulares
- Bordas bem definidas
- Padrões de plantio visíveis

### 7.3 Validação

Para validar suas interpretações:
1. Compare com mapas de uso do solo existentes
2. Verifique histórico da área
3. Use conhecimento local quando possível
4. Considere a sazonalidade agrícola da região